In [74]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options  
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from bs4 import BeautifulSoup
import pandas as pd
import yfinance as yf
import numpy as np
from datetime import datetime, timedelta
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import chi2
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import seaborn as sns

In [75]:
number_of_scrolls = 40
link = "https://cryptorank.io/news/tag/bitcoin"

sources = {
    "unselected": [
        "AMBCrypto",
        "Bitcoin Magazine",
        "Bitcoinist",
        "Consensys",
        "Crunchbase",
        "CryptoPolitan",
        "Forkast",
        "NewsBTC",
        "The Daily Hodl",
        "U.Today",
        "Watcher.Guru",
        "ZyCrypto",
        "Bitcoin Sistemi",
        "BTC-Pulse",
        "CoinCu News",
        "Techcrunch",
        "CoinEdition",
        "Coingape",
        "Coinpedia",
        "COINTURK NEWS",
        "Cryptonews",
        "CryptoPotato",
        "Bitcoin News",
        "CryptoTicker",
        "DroomDroom",
    ],
    "selected": [
        "Cointelegraph",
        "CoinDesk",
        "Crypto Daily",
        "CryptoSlate",
    ]
}

In [76]:
def WebScrapping(): 
    chrome_options = Options()
    # chrome_options.add_argument("--headless") 

    driver = webdriver.Chrome(options=chrome_options)

    driver.get(link)

    time.sleep(2)
    wait = WebDriverWait(driver, 10)
    SCROLL_PAUSE_TIME = 5

    button = driver.find_element(By.CLASS_NAME, "sc-1b84392e-0")
    button.click()
    time.sleep(1)

    change_select_button = driver.find_element(By.XPATH, "/html/body/div[6]/div/div/div[2]/div[2]/div[2]/button[1]")
    apply_button = driver.find_element(By.XPATH, "/html/body/div[6]/div/div/div[2]/div[3]/button[2]")
    time.sleep(1)
    change_select_button.click()

    content = driver.find_element(By.XPATH, "/html/body/div[6]/div/div/div[2]/div[2]/div[1]/div[2]/div")
    texts = content.find_elements(By.CLASS_NAME, "sc-3825f359-5")
    
    for text in texts:
        filtered = filter(lambda source: source == text.text, sources["unselected"])

        if len(list(filtered)) == 0:
            text.click()
    
    cookies = driver.find_element(By.XPATH, '//*[@id="CookiesPopup"]/div/button')
    cookies.click()
    
    apply_button.click()

    trading = driver.find_element(By.XPATH, '//*[@id="root-container"]/div[1]/div/div[3]/div/div/a[9]')
    trading.click()
    time.sleep(1)

    last_height = driver.execute_script("return document.body.scrollHeight")
    step_scroll = 0

    while step_scroll <= number_of_scrolls:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight - 1000);")
        time.sleep(SCROLL_PAUSE_TIME)
        new_height = driver.execute_script("return document.body.scrollHeight - 1000")
        if new_height == last_height:
            break
        last_height = new_height
        step_scroll += 1

    html = driver.page_source

    driver.quit()

    return html

site = BeautifulSoup(WebScrapping(), "html.parser")

In [77]:
container = site.find("div", class_="sc-3a1bd8cb-0 hqCBiY")
boxes = container.find_all("div", class_="sc-da92bc0b-3 gUpZWz")

json_boxes = []

for box in boxes:
    title = box.find_all("h2", class_="sc-50f3633f-0 sc-da92bc0b-2 kqEOSA kAtqpd")[0].text

    subtitle = box.find_all("div", class_="sc-da92bc0b-6 eVVqXG")[0].text
    
    date_box = box.find_all("div", class_="sc-363256ef-0 LgVuA")[0]
    date = box.find_all("div", class_="sc-363256ef-1 ciRkfL")[0].text

    scrap_data = {
        "Date": date,
        "Title": title,
        "Subtitle": subtitle
    }

    json_boxes.append(scrap_data)

data = pd.DataFrame(json_boxes)

In [78]:
data_t = data

def corrigir_data(data_string):
    data_objeto = datetime.strptime(str(data_string), "%b, %d, %Y")

    return data_objeto.strftime("%Y-%m-%d")

data_t['Date'] = data_t['Date'].apply(corrigir_data)
data_t['Date'] = pd.to_datetime(data_t['Date'], format="%Y-%m-%d")
data_t = data_t.sort_values(by="Date")
data_t = data_t.reset_index().drop(columns=["index"])
data_t["Text"] = data_t["Title"] + data_t["Subtitle"]
data_t = data_t.groupby('Date')[["Text"]].agg(lambda x: ' and '.join(x)).reset_index()

data_t

,Date,Text
0,2023-10-25,Bitcoin Retreats From $35K; Miner Selling Pres...
1,2023-10-26,"Dogecoin, Shiba Inu Jump 9% as Crypto Traders ..."
2,2023-10-27,Bitcoin Surges to All-Time Highs in Turkey and...
3,2023-10-30,Tokenized U.S. Treasury Market Grows Nearly 60...
4,2023-10-31,Blockchain Startup Etherfuse Rolls Out Tokeniz...
...,...,...
105,2024-03-11,Saylor says Bitcoin will ‘eat gold’ in the com...
106,2024-03-12,Coinbase looks to raise $1 billion via bond of...
107,2024-03-13,Why Prime Brokers Could Be a Source of Crypto ...
108,2024-03-14,"Bitcoin Slumps 4%, Falling Below $70K; Solana ..."


In [79]:
data_atual = datetime.now().date()
data_final = data_atual - timedelta(days=0)

btc = yf.download("BTC-USD", end=data_final)
btc["Return"] = btc["Close"].pct_change()
btc["Return_D1"] = btc["Return"].shift(-1)
btc = btc.dropna()
btc = btc.reset_index()
btc = btc.sort_index(ascending=False)
btc = btc.reset_index().drop(columns=["index", "Adj Close"])

[*********************100%%**********************]  1 of 1 completed


In [80]:
df = pd.merge(data_t, btc, on="Date")
df = df.sort_index(ascending=True)
df["Date"] = pd.to_datetime(df["Date"], unit="sm")
df = df.set_index("Date")

df["Direction_D1"] = np.where(df["Return_D1"] < 0, "Negative", "Positive")
df["Direction_D1_Fact"] = df["Direction_D1"].factorize()[0]

df.head()

,Text,Open,High,Low,Close,Volume,Return,Return_D1,Direction_D1,Direction_D1_Fact
Date,,,,,,,,,,
2023-10-25,Bitcoin Retreats From $35K; Miner Selling Pres...,33916.042969,35133.757812,33709.109375,34502.820312,25254318008,0.017736,-0.010033,Negative,0
2023-10-26,"Dogecoin, Shiba Inu Jump 9% as Crypto Traders ...",34504.289062,34832.910156,33762.324219,34156.648438,19427195376,-0.010033,-0.007227,Negative,0
2023-10-27,Bitcoin Surges to All-Time Highs in Turkey and...,34156.500000,34238.210938,33416.886719,33909.800781,16418032871,-0.007227,0.005302,Positive,1
2023-10-30,Tokenized U.S. Treasury Market Grows Nearly 60...,34531.742188,34843.933594,34110.972656,34502.363281,17184860315,-0.001046,0.004794,Positive,1
2023-10-31,Blockchain Startup Etherfuse Rolls Out Tokeniz...,34500.078125,34719.253906,34083.308594,34667.781250,15758270810,0.004794,0.022196,Positive,1


In [81]:
from transformers import pipeline

pipe = pipeline("text-classification", model="mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis")

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaForSequenceClassification: ['roberta.embeddings.position_ids']
- This IS expected if you are initializing TFRobertaForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFRobertaForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.


In [82]:
def process_pipe(text):
    return pipe(text[:500])[0]["label"]

In [83]:
df["Sentiment"] = df["Text"].map(process_pipe)
df["Sentiment_Fact"] = df["Sentiment"].factorize()[0]

In [84]:
df

,Text,Open,High,Low,Close,Volume,Return,Return_D1,Direction_D1,Direction_D1_Fact,Sentiment,Sentiment_Fact
Date,,,,,,,,,,,,
2023-10-25,Bitcoin Retreats From $35K; Miner Selling Pres...,33916.042969,35133.757812,33709.109375,34502.820312,25254318008,0.017736,-0.010033,Negative,0,positive,0
2023-10-26,"Dogecoin, Shiba Inu Jump 9% as Crypto Traders ...",34504.289062,34832.910156,33762.324219,34156.648438,19427195376,-0.010033,-0.007227,Negative,0,positive,0
2023-10-27,Bitcoin Surges to All-Time Highs in Turkey and...,34156.500000,34238.210938,33416.886719,33909.800781,16418032871,-0.007227,0.005302,Positive,1,positive,0
2023-10-30,Tokenized U.S. Treasury Market Grows Nearly 60...,34531.742188,34843.933594,34110.972656,34502.363281,17184860315,-0.001046,0.004794,Positive,1,positive,0
2023-10-31,Blockchain Startup Etherfuse Rolls Out Tokeniz...,34500.078125,34719.253906,34083.308594,34667.781250,15758270810,0.004794,0.022196,Positive,1,positive,0
...,...,...,...,...,...,...,...,...,...,...,...,...
2024-03-10,"BlackRock's Bitcoin ETF Nears 200K BTC, Passin...",68500.257812,70005.203125,68239.976562,69019.789062,35683977532,0.007605,0.044974,Positive,1,neutral,2
2024-03-11,Saylor says Bitcoin will ‘eat gold’ in the com...,69020.546875,72850.710938,67194.882812,72123.906250,65716656765,0.044974,-0.008910,Negative,0,positive,0
2024-03-12,Coinbase looks to raise $1 billion via bond of...,72125.125000,72825.656250,68728.851562,71481.289062,62554434520,-0.008910,0.022414,Positive,1,positive,0
